In [3]:
import pandas as pd
p = "/Users/estellekim/Downloads/web_routineness_release/raw/browsing.csv"
df = pd.read_csv(p)
df[df["panelist_id"].between(0, 7)].to_csv(p.replace(".csv","_panelist0_7.csv"), index=False)


In [5]:
%pip install numpy
import numpy as np

df = pd.read_csv("/Users/estellekim/Downloads/web_routineness_release/raw/browsing_panelist0_7.csv")  # has columns: domain, subdomain

sub = df["subdomain"].astype(str).str.strip().str.strip(".")
dom = df["domain"].astype(str).str.strip().str.strip(".")

df["full_domain"] = np.where(sub.eq("") | sub.eq("nan"),
                             dom,
                             sub + "." + dom)

df.to_csv("output.csv", index=False)


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd

IN = "output.csv"
df = pd.read_csv(IN)

# Parse the timestamp column
df["used_at_dt"] = pd.to_datetime(df["used_at"], errors="coerce")

# If these timestamps are UTC (common), set tz:
df["used_at_utc"] = df["used_at_dt"].dt.tz_localize("UTC")  # drop this if already tz-aware

# If they are local time in, say, America/New_York, localize instead:
# df["used_at_local"] = pd.to_datetime(df["used_at"], errors="coerce").dt.tz_localize("America/New_York")

# Derivatives
df["used_at_iso"]   = df["used_at_utc"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")     # ISO-8601
df["used_at_unix"]  = df["used_at_utc"].astype("int64") // 10**9              # epoch seconds
df["date"]          = df["used_at_utc"].dt.date
df["hour"]          = df["used_at_utc"].dt.hour
df["weekday"]       = df["used_at_utc"].dt.day_name()

# If you computed end_time earlier and want epoch too:
# df["end_time_unix"] = df["end_time"].dt.tz_localize("UTC").astype("int64") // 10**9

print(df[["used_at","used_at_iso","used_at_unix","date","hour","weekday"]].head())


               used_at           used_at_iso  used_at_unix        date  hour  \
0  2018-10-05 01:48:03  2018-10-05T01:48:03Z    1538704083  2018-10-05     1   
1  2018-10-05 01:38:57  2018-10-05T01:38:57Z    1538703537  2018-10-05     1   
2  2018-10-05 00:06:27  2018-10-05T00:06:27Z    1538697987  2018-10-05     0   
3  2018-10-05 00:00:27  2018-10-05T00:00:27Z    1538697627  2018-10-05     0   
4  2018-10-05 01:08:23  2018-10-05T01:08:23Z    1538701703  2018-10-05     1   

  weekday  
0  Friday  
1  Friday  
2  Friday  
3  Friday  
4  Friday  


In [12]:
import pandas as pd

IN  = "output.csv"                               # your original file
OUT = IN.replace(".csv", "_collapsed_iso.csv")

USER = "panelist_id"
TS   = "used_at"
DUR  = "active_seconds"
FDOM = "full_domain"

# Load & parse
df = pd.read_csv(IN)
df[TS]  = pd.to_datetime(df[TS], errors="coerce")
df[DUR] = pd.to_numeric(df[DUR], errors="coerce").fillna(0).astype(float)

# Sort so "consecutive" has a clear meaning (per user, chronological)
df = df.sort_values([USER, TS], kind="mergesort").reset_index(drop=True)

# Per-row end time (used for true run end)
df["_row_end"] = df[TS] + pd.to_timedelta(df[DUR], unit="s")

# New run starts when user or domain changes from previous row
is_new_run = (df[USER].ne(df[USER].shift())) | (df[FDOM].ne(df[FDOM].shift()))
df["_run_id"] = is_new_run.cumsum()

# Aggregate each consecutive run
collapsed = (
    df.groupby([USER, FDOM, "_run_id"], as_index=False)
      .agg(
          start_time=(TS, "first"),
          end_time=("_row_end", "max"),
          total_active_seconds=(DUR, "sum"),
          row_count=("_run_id", "size"),
      )
)

# Replace time columns with ISO-8601 UTC strings (easy for frontend)
collapsed["start_time"] = collapsed["start_time"].dt.tz_localize("UTC").dt.strftime("%Y-%m-%dT%H:%M:%SZ")
collapsed["end_time"]   = collapsed["end_time"].dt.tz_localize("UTC").dt.strftime("%Y-%m-%dT%H:%M:%SZ")

# Final columns and sort by person then time
collapsed = collapsed[[USER, FDOM, "start_time", "end_time", "total_active_seconds", "row_count"]]
collapsed = collapsed.sort_values([USER, "start_time"], kind="mergesort").reset_index(drop=True)

collapsed.to_csv(OUT, index=False)
print(f"Wrote {OUT} ({len(collapsed)} rows)")


Wrote output_collapsed_iso.csv (10482 rows)


In [ ]:
#!/usr/bin/env python3
"""
Generate static JSON artifacts for the Internet Atlas frontend.

Inputs:
  - browsing CSV with columns:
      panelist_id, used_at, active_seconds, domain, subdomain, full_domain

Outputs (under --out /jsons):
  - edges_u0_7.json
  - edge_users_u0_7.json
  - user_edges_u0_7/<userId>.json
  - node_stats_u0_7.json

Author: you
"""

import argparse
import json
import os
from pathlib import Path
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np


# ------------------------- Config -------------------------

USER_MIN = 0
USER_MAX = 7   # inclusive; 0..7 -> 8 users total (match file suffix u0_7)

# Output filenames (match your static API expectations)
EDGES_FILE = "edges_u0_7.json"
EDGE_USERS_FILE = "edge_users_u0_7.json"
USER_EDGES_DIR = "user_edges_u0_7"
NODE_STATS_FILE = "node_stats_u0_7.json"


# ------------------------- Helpers -------------------------

def iso_utc(dt: pd.Series) -> pd.Series:
    """Convert pandas datetime to ISO-8601 Z strings."""
    # Localize naive to UTC (adjust if your data is already tz-aware)
    if dt.dt.tz is None:
        dt = dt.dt.tz_localize("UTC")
    return dt.dt.strftime("%Y-%m-%dT%H:%M:%SZ")


def normalize_domain(s: str) -> str:
    return (s or "").strip().lower()


def ensure_cols(df: pd.DataFrame, cols: List[str]):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")


# ------------------------- Core steps -------------------------

def load_and_filter(input_csv: str) -> pd.DataFrame:
    """Load CSV, parse/clean, filter users."""
    df = pd.read_csv(input_csv)

    # required columns
    ensure_cols(df, ["panelist_id", "used_at", "active_seconds"])

    # full_domain: build if not present from domain + subdomain
    if "full_domain" not in df.columns:
        ensure_cols(df, ["domain"])
        sub = df["subdomain"] if "subdomain" in df.columns else ""
        df["full_domain"] = (
            sub.fillna("").astype(str).str.strip().str.lower().replace("", np.nan)
            + "."
            + df["domain"].astype(str).str.strip().str.lower()
        )
        # Remove leading '.' when subdomain missing
        df["full_domain"] = df["full_domain"].str.replace("^\\.", "", regex=True)

    # normalize critical fields
    df["full_domain"] = df["full_domain"].astype(str).apply(normalize_domain)
    df["panelist_id"] = pd.to_numeric(df["panelist_id"], errors="coerce").astype("Int64")
    df["active_seconds"] = pd.to_numeric(df["active_seconds"], errors="coerce").fillna(0).astype(float)
    df["used_at"] = pd.to_datetime(df["used_at"], errors="coerce")

    # filter desired users
    df = df[df["panelist_id"].between(USER_MIN, USER_MAX)].copy()

    # drop rows with missing essentials
    df = df.dropna(subset=["panelist_id", "used_at", "full_domain"]).reset_index(drop=True)
    return df


def collapse_sessions(df: pd.DataFrame) -> pd.DataFrame:
    """
    Collapse consecutive rows per user+domain into sessions:
      start_time = first used_at
      end_time   = max(used_at + active_seconds) within the run
      total_active_seconds = sum(active_seconds) in the run
    """
    USER, TS, DUR, FDOM = "panelist_id", "used_at", "active_seconds", "full_domain"

    df = df.sort_values([USER, TS], kind="mergesort").reset_index(drop=True)
    df["_row_end"] = df[TS] + pd.to_timedelta(df[DUR], unit="s")

    is_new_run = (df[USER].ne(df[USER].shift())) | (df[FDOM].ne(df[FDOM].shift()))
    df["_run_id"] = is_new_run.cumsum()

    collapsed = (
        df.groupby([USER, FDOM, "_run_id"], as_index=False)
          .agg(
              start_time=(TS, "first"),
              end_time=("_row_end", "max"),
              total_active_seconds=(DUR, "sum"),
              row_count=("_run_id", "size"),
          )
    )
    # Convert to ISO-8601 Z
    collapsed["start_time"] = iso_utc(collapsed["start_time"])
    collapsed["end_time"]   = iso_utc(collapsed["end_time"])

    # Final columns
    collapsed = collapsed[[USER, FDOM, "start_time", "end_time", "total_active_seconds", "row_count"]]
    return collapsed


def build_user_edges_from_sessions(sessions: pd.DataFrame) -> Dict[int, List[Tuple[str, str]]]:
    """
    Build per-user edge sequences from sessions:
    For each user, sort sessions by start_time and add a transition from
    previous session's domain to current session's domain when it changes.
    Returns: { userId: [(origin, target), ...] }
    """
    USER, FDOM = "panelist_id", "full_domain"

    # parse times back (for reliable sorting)
    tmp = sessions.copy()
    tmp["_start_dt"] = pd.to_datetime(tmp["start_time"], utc=True, errors="coerce")

    user_edges: Dict[int, List[Tuple[str, str]]] = {}
    for user_id, g in tmp.groupby(USER, sort=True):
        g = g.sort_values("_start_dt", kind="mergesort")
        domains = g[FDOM].tolist()
        edges = []
        prev = None
        for d in domains:
            if prev is not None and d != prev:
                edges.append((prev, d))
            prev = d
        user_edges[int(user_id)] = edges
    return user_edges


def aggregate_edges(user_edges: Dict[int, List[Tuple[str, str]]]) -> Tuple[List[Dict], Dict[str, List[int]]]:
    """
    Aggregate edges across users.
    Returns:
      - edges list with id, origin, target, num_users
      - edge -> [userIds] map
    """
    edge_to_users: Dict[Tuple[str, str], set] = {}
    for uid, edges in user_edges.items():
        for (o, t) in edges:
            key = (normalize_domain(o), normalize_domain(t))
            s = edge_to_users.setdefault(key, set())
            s.add(uid)

    # assign ids and build edges list
    edges_list = []
    edge_users_map: Dict[str, List[int]] = {}
    eid = 1
    for (o, t), users in edge_to_users.items():
        edges_list.append({
            "id": eid,
            "origin": o,
            "target": t,
            "num_users": len(users)
        })
        edge_users_map[f"{o}|{t}"] = sorted(int(u) for u in users)
        eid += 1

    # stable sort edges (by num_users desc then alpha)
    edges_list.sort(key=lambda e: (-e["num_users"], e["origin"], e["target"]))
    return edges_list, edge_users_map


def build_node_stats_from_sessions(sessions: pd.DataFrame) -> Dict:
    """
    Quick domain totals (used for overlay). For simplicity we compute domain-level
    totals independent of mode, and populate both by_origin and by_target the same way.
    """
    FDOM = "full_domain"

    tmp = sessions.copy()
    tmp["domain"] = tmp[FDOM].apply(normalize_domain)

    grp = tmp.groupby("domain", as_index=False).agg(
        visit_count=("domain", "size"),
        total_time_spent=("total_active_seconds", "sum"),
    )
    grp["avg_time_per_visit"] = grp["total_time_spent"] / grp["visit_count"]

    # materialize dicts
    stats_map = {
        row["domain"]: {
            "visit_count": int(row["visit_count"]),
            "total_time_spent": float(row["total_time_spent"]),
            "avg_time_per_visit": float(row["avg_time_per_visit"]),
        }
        for _, row in grp.iterrows()
    }

    return {
        "by_origin": stats_map,
        "by_target": stats_map,  # same totals (you can split by edge-role later if desired)
    }


# ------------------------- Orchestrator -------------------------

def main():
    input = "browsing.csv"
    out_dir =  "new.csv"
    out_dir.mkdir(parents=True, exist_ok=True)
    (out_dir / USER_EDGES_DIR).mkdir(parents=True, exist_ok=True)

    print("• Loading…")
    df = load_and_filter(input)

    print("• Collapsing sessions…")
    sessions = collapse_sessions(df)

    # If you also want to export sessions for inspection, uncomment:
    # (out_dir / "sessions_u0_7.json").write_text(json.dumps({
    #     "results_count": int(len(sessions)),
    #     "results": sessions.to_dict(orient="records")
    # }, ensure_ascii=False))

    print("• Building per-user edges…")
    user_edges = build_user_edges_from_sessions(sessions)

    print("• Aggregating edges…")
    edges_list, edge_users_map = aggregate_edges(user_edges)

    print("• Computing node stats…")
    node_stats = build_node_stats_from_sessions(sessions)

    # ---- Write files ----
    print("• Writing edges file:", EDGES_FILE)
    with open(out_dir / EDGES_FILE, "w", encoding="utf-8") as f:
        json.dump({
            "results_count": len(edges_list),
            "results": edges_list
        }, f, ensure_ascii=False)

    print("• Writing edge users map:", EDGE_USERS_FILE)
    with open(out_dir / EDGE_USERS_FILE, "w", encoding="utf-8") as f:
        json.dump(edge_users_map, f, ensure_ascii=False)

    print("• Writing per-user edge sequences:", USER_EDGES_DIR)
    for uid, edges in user_edges.items():
        rows = [{
            "id": i + 1,
            "origin": o,
            "target": t,
            "num_users": 1  # per-user sequence: single user
        } for i, (o, t) in enumerate(edges)]
        with open(out_dir / USER_EDGES_DIR / f"{uid}.json", "w", encoding="utf-8") as f:
            json.dump({
                "results_count": len(rows),
                "results": rows
            }, f, ensure_ascii=False)

    print("• Writing node stats:", NODE_STATS_FILE)
    with open(out_dir / NODE_STATS_FILE, "w", encoding="utf-8") as f:
        json.dump(node_stats, f, ensure_ascii=False)

    print("✓ Done.")


if __name__ == "__main__":
    main()
